
<h1 id="(Optional)-웹-크롤링2---Dynamic-Crawling">(Optional) 웹 크롤링2 - Dynamic Crawling<a class="anchor-link" href="#(Optional)-웹-크롤링2---Dynamic-Crawling">¶</a></h1>



<h1 id="0.-라이브러리">0. 라이브러리<a class="anchor-link" href="#0.-라이브러리">¶</a></h1>


In [2]:
# headless 옵션을 통하여 창을 안띄우고 스크래핑하는 것도 가능함
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
import time




<h1 id="1.-Selenium-기초">1. Selenium 기초<a class="anchor-link" href="#1.-Selenium-기초">¶</a></h1><p>자신의 크롬 버전을 확인하고 크롬 <a href="https://chromedriver.chromium.org/downloads">웹드라이버</a>를 다운받아놓아야합니다.</p>
<ul>
<li>2020.09.13 기준 최신 버전: <code>85.0.4183.102</code></li>
</ul>



<h3 id="1.1.-Simple-Text-Crawling">1.1. Simple Text Crawling<a class="anchor-link" href="#1.1.-Simple-Text-Crawling">¶</a></h3><p>멜론 사이트에서 노래 제목을 크롤링해보자</p>
<p>URL: <a href="https://www.melon.com/chart/index.htm">https://www.melon.com/chart/index.htm</a></p>


In [4]:
# 크롬드라이버 경로 지정
DRIVER_PATH = 'C:/selenium/chromedriver.exe'


In [5]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH) 
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

# 해당 url로 접속함
driver.get(url)

# 이 부분 코드는 뒤에서 쓰이지 않음
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# title crawling
# WebDriverWait를 통해서 당장 엘리먼트가 나타나지 않아도 20초까지 기다릴 수 있게 함
title = WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.CSS_SELECTOR, "#frm > div > table > tbody > tr:nth-child(1) > td:nth-child(4) > div > div")))

# print("Title: {}".format(title.text))

title.text

# 윈도우에서 브라우저 창이 누적되는 것을 방지하기 위하여 사용함
driver.close()

'Dynamite\n방탄소년단'


<p>css selector의 <strong><code>규칙</code></strong>을 찾아본다</p>
<ul>
<li>1번째 제목: #frm &gt; div &gt; table &gt; tbody &gt; tr:nth-child(<strong>1</strong>) &gt; td:nth-child(4) &gt; div &gt; div"</li>
<li>2번째 제목: #frm &gt; div &gt; table &gt; tbody &gt; tr:nth-child(<strong>2</strong>) &gt; td:nth-child(4) &gt; div &gt; div</li>
</ul>
<p>.
.
.</p>
<ul>
<li>100번째 제목: #frm &gt; div &gt; table &gt; tbody &gt; tr:nth-child(<strong>100</strong>) &gt; td:nth-child(4) &gt; div &gt; div</li>
</ul>
<hr/>
<p>또는 <code>XPATH</code>로도 확인해보자 (<code>full Xpath</code>)</p>
<ul>
<li>1번째 제목: //*[@id="frm"]/div/table/tbody/tr[<strong>1</strong>]/td[4]/div/div</li>
<li>2번째 제목: //*[@id="frm"]/div/table/tbody/tr[<strong>2</strong>]/td[4]/div/div</li>
</ul>
<p>.
.
.</p>
<ul>
<li>50번째 제목: //*[@id="frm"]/div/table/tbody/tr[<strong>100</strong>]/td[4]/div/div</li>
</ul>


In [6]:

# 2번째 제목 크롤링
WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.XPATH, "//*[@id='frm']/div/table/tbody/tr[2]/td[4]/div/div"))).text



'취기를 빌려 (취향저격 그녀 X 산들)\n산들'


<h3 id="1.2.-Text-Crawling-with-for-loop">1.2. Text Crawling with for loop<a class="anchor-link" href="#1.2.-Text-Crawling-with-for-loop">¶</a></h3><p>위에서 찾은 Xpath의 규칙을 바탕으로 for loop 만들자</p>


In [7]:

 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)

# 이 부분 코드는 뒤에서 쓰이지 않음
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []

# title crawling (TOP 50)
for i in range(1, 51):
    title = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f"//*[@id='frm']/div/table/tbody/tr[{i}]/td[4]/div/div")))
    title_list.append(title.text)
    
print(title_list)

# 윈도우에서 브라우저 창이 누적되는 것을 방지하기 위하여 사용함
driver.close()

['Dynamite\n방탄소년단', '취기를 빌려 (취향저격 그녀 X 산들)\n산들', '눈누난나 (NUNU NANA)\n제시 (Jessi)', '마리아 (Maria)\n화사 (Hwa Sa)', '다시 여기 바닷가\n싹쓰리 (유두래곤, 린다G, 비룡)', 'When We Disco (Duet with 선미)\n박진영', '오래된 노래\n스탠딩 에그', 'How You Like That\nBLACKPINK', '에잇(Prod.&Feat. SUGA of BTS)\n아이유', '내 마음이 움찔했던 순간 (취향저격 그녀 X 규현)\n규현 (KYUHYUN)', '아로하\n조정석', 'Dolphin\n오마이걸 (OH MY GIRL)', 'Downtown Baby\n블루 (BLOO)', '어떻게 지내 (Prod. By VAN.C)\n오반', '홀로\n이하이', 'Summer Hate (Feat. 비)\n지코 (ZICO)', 'Blueming\n아이유', 'Dance Monkey\nTones And I', '덤디덤디 (DUMDi DUMDi)\n(여자)아이들', '흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야\n장범준', 'Memories\nMaroon 5', '살짝 설렜어 (Nonstop)\n오마이걸 (OH MY GIRL)', 'METEOR\n창모 (CHANGMO)', 'Not Shy\nITZY (있지)', '어떻게 이별까지 사랑하겠어, 널 사랑하는 거지\nAKMU (악동뮤지션)', '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)\n방탄소년단', '사랑은 지날수록 더욱 선명하게 남아\n전상근', '사랑하게 될 줄 알았어\n전미도', '늦은 밤 너의 집 앞 골목길에서\n노을', 'Ice Cream (with Selena Gomez)\nBLACKPINK', '그 여름을 틀어줘\n싹쓰리 (유두래곤, 린다G, 비룡)', '거짓말이라도 해서 널 보고싶어\n백지영', '보라빛 밤 (pporappippam)\n선미', '모든 날, 모든 순간 (Every day


<p>나중에 필요한 변수(제목, 가수, 가사... 등)을 모두 긁어 한번에 <strong>데이터프레임</strong>으로 저장하여 보관한다!</p>



<h3 id="1.3.-Text-Crawling-(Click-&amp;-Back)">1.3. Text Crawling (Click &amp; Back)<a class="anchor-link" href="#1.3.-Text-Crawling-(Click-&amp;-Back)">¶</a></h3><p>클릭하고 나오기 -&gt; 동적 크롤링 가능 (<strong><code>가사</code></strong> 크롤링 가능)</p>
<p>노래 제목에 링크가 걸려있기 때문에, 해당 링크까지의 <strong>XPath</strong>를 추가한다.</p>


In [8]:

 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)

# 이 부분 코드는 뒤에서 쓰이지 않음
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 1번째 click하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[3]/div/a')))
click_element.click()    

# back
driver.back()


# 2번째 click하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[2]/td[3]/div/a')))
click_element.click()    

# back
driver.back()

# 윈도우에서 브라우저 창이 누적되는 것을 방지하기 위하여 사용함
driver.close()


<h3 id="1.4.-Text-Crawling-including-contents">1.4. Text Crawling including contents<a class="anchor-link" href="#1.4.-Text-Crawling-including-contents">¶</a></h3><ul>
<li><p><strong>1.2</strong>처럼 for문과 함께 써보자! (첫 페이지 5개의 글에 대해 <code>title</code>, <code>artist</code>, <code>heart</code>(하트 갯수), <code>lyrics</code>(가사)를 크롤링</p>
</li>
<li><p><strong>1.3</strong>에서 사용한 click &amp; back을 활용하자</p>
</li>
</ul>


In [11]:

 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"
driver.get(url)

# 이 부분 코드는 뒤에서 쓰이지 않음
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []
artist_list = []
heart_list = []
lyrics_list = []

# crawling (TOP 5)
for i in range(1, 6):
    # click
    click_element = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{i}]/td[3]/div/a')))
    click_element.click()

    # title crawling
    title = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.song_name")))
    title_list.append(title.text)

    # artist crawling
    artist = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)")))
    artist_list.append(artist.text)
    
    # heart crawling
    heart = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#d_like_count")))
    heart_list.append(heart.text)

    # lyrics crawling
    lyrics = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#d_video_summary")))
    lyrics_list.append(lyrics.text)
    
    # back
    driver.back()
    
print(title_list)
print(artist_list)
print(heart_list)
print(lyrics_list)

# 윈도우에서 브라우저 창이 누적되는 것을 방지하기 위하여 사용함
driver.close()

['Dynamite', '취기를 빌려 (취향저격 그녀 X 산들)', '눈누난나 (NUNU NANA)', '마리아 (Maria)', '다시 여기 바닷가']
['방탄소년단', '산들', '제시 (Jessi)', '화사 (Hwa Sa)', '싹쓰리 (유두래곤, 린다G, 비룡)']
['278,688', '103,391', '76,553', '140,069', '222,456']
["Cos ah ah\nI’m in the stars tonight\nSo watch me bring the fire\nand set the night alight\nShoes on get up in the morn\nCup of milk let’s rock and roll\nKing Kong kick the drum\nrolling on like a rolling stone\nSing song when I’m walking home\nJump up to the top LeBron\nDing dong call me on my phone\nIce tea and a game of ping pong\nThis is getting heavy\nCan you hear the bass boom\nI’m ready\nLife is sweet as honey\nYeah this beat cha ching\nlike money\nDisco overload I’m into\nthat I’m good to go\nI'm diamond you know I glow up\nHey so let’s go\nCos ah ah\nI’m in the stars tonight\nSo watch me bring the fire\nand set the night alight\nShining through the city\nwith a little funk and soul\nSo I’mma light it up\nlike dynamite woah\nBring a friend join the crowd\nWhoever wanna co


<h3 id="TIP:-보통은-결과값을-데이터프레임-형태로-저장한다">TIP: 보통은 결과값을 <code>데이터프레임</code> 형태로 저장한다<a class="anchor-link" href="#TIP:-보통은-결과값을-데이터프레임-형태로-저장한다">¶</a></h3>


In [12]:

# 결과 변수
raw_result = {'title': title_list,
              'artist': artist_list,
              'heart': heart_list,
          'lyrics': lyrics_list}

result = pd.DataFrame(raw_result)

# # csv 파일로 save
# result.to_csv("MelonTop5", mode='w')

# driver 종료

driver.quit()

In [13]:

result



,title,artist,heart,lyrics
0,Dynamite,방탄소년단,"278,688",Cos ah ah\nI’m in the stars tonight\nSo watch ...
1,취기를 빌려 (취향저격 그녀 X 산들),산들,"103,391",언제부턴가 불쑥\n내 습관이 돼버린 너\n혹시나 이런 맘이\n어쩌면 부담일까\n널 ...
2,눈누난나 (NUNU NANA),제시 (Jessi),"76,553",I’m trying to give u\nsomething more\nSo come ...
3,마리아 (Maria),화사 (Hwa Sa),"140,069",욕을 하도 먹어 체했어 하도\n서러워도 어쩌겠어 I do\n모두들 미워하느라 애썼네...
4,다시 여기 바닷가,"싹쓰리 (유두래곤, 린다G, 비룡)","222,456",예아 호우 예예예\n싹쓰리 인더 하우스\n커커커커커몬 싹 쓰리 투 렛츠고\n나 다시...



<p><strong><code>데이터프레임</code></strong> 형식을 이용하면, 가독성도 좋고, 나중에 데이터 핸들링하기에도 편리하다!</p>



<h1 id="2.-Image-Crawling">2. Image Crawling<a class="anchor-link" href="#2.-Image-Crawling">¶</a></h1><p>이미지 크롤링하기</p>
<ul>
<li>1번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[<strong>1</strong>]/td[4]/div/a/img</li>
<li>2번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[<strong>2</strong>]/td[4]/div/a/img</li>
</ul>
<p>...</p>
<ul>
<li>50번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[<strong>50</strong>]/td[4]/div/a/img</li>
</ul>



<h4 id="STEP1.-URL-Crawling">STEP1. URL Crawling<a class="anchor-link" href="#STEP1.-URL-Crawling">¶</a></h4>


In [14]:

 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)

# 이 부분 코드는 뒤에서 쓰이지 않음
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
link_list = []

# # img crawling (TOP 50)
for i in range(1, 51):
    
    img = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.CSS_SELECTOR, f"#frm > div > table > tbody > tr:nth-child({i}) > td:nth-child(2) > div > a > img")))

    link_list.append(img.get_attribute('src'))

print(link_list)

# 윈도우에서 브라우저 창이 누적되는 것을 방지하기 위하여 사용함
driver.close()

['https://cdnimg.melon.co.kr/cm2/album/images/104/79/150/10479150_20200918102847_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/63/600/10463600_20200720152905_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/69/416/10469416_20200730151034_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/52/351/10452351_20200629152036_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/62/799/10462799_20200717150822_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/75/061/10475061_20200812120927_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm/album/images/021/48/596/2148596_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/51/566/10451566_20200626114914_500.jpg/melon/resize/120/quality/80/optimize', 


<h4 id="STEP2.-Download-images-using-URLs">STEP2. Download images using URLs<a class="anchor-link" href="#STEP2.-Download-images-using-URLs">¶</a></h4><p>자신의 디렉토리에 <code>img</code> 폴더 생성하고 실행</p>


In [16]:

import urllib.request

count = 0
for link in link_list:
    count += 1
    urllib.request.urlretrieve(link, './img/img' + str(count) + '.jpg')



# 추가 크롤링 (네이버 급상승검색어 스크래핑)

## 1. 페이지 접속

In [57]:
driver = webdriver.Chrome(DRIVER_PATH) 
driver.implicitly_wait(2)

url = "https://www.naver.com/"

# 해당 url로 접속함
driver.get(url)

## 2. 검색어 설정

In [58]:
# 분야별 설정
for i in range(1, 6):
    radio_bt = WebDriverWait(driver, 2) \
      .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="NM_RTK_VIEW_filter_wrap"]/li[{str(i)}]/div/a[1]')))

    # 페이지에서 해당 엘리먼트가 브라우저 창에 직접 보여지지 않은 상태에서도 클릭하기 위하여 자바스크립트를 이용함
    driver.execute_script("arguments[0].click();", radio_bt)
    
# 전체연령 설정
all_age_bt = WebDriverWait(driver, 2) \
      .until(EC.presence_of_element_located((By.XPATH, '//*[@id="NM_RKT_VIEW_filter_age_wrap"]/li[6]/a')))
driver.execute_script("arguments[0].click();", all_age_bt)

# 설정 완료
completion_bt = WebDriverWait(driver, 2) \
      .until(EC.presence_of_element_located((By.XPATH, '//*[@id="NM_RTK_VIEW_set_btn"]')))
driver.execute_script("arguments[0].click();", completion_bt)

## 3. 스크래핑

In [59]:
# 더 보기
more_see = WebDriverWait(driver, 2) \
      .until(EC.presence_of_element_located((By.XPATH, '//*[@id="NM_RTK_ROLLING_WRAP"]')))
more_see.click()

# 급상승검색어 스크래핑
real_time_keywords = WebDriverWait(driver, 5) \
  .until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".list_realtime span.keyword")))

real_time_keywords = [real_time_keyword.text for real_time_keyword in real_time_keywords if real_time_keyword.text]

for real_time_keyword in real_time_keywords:
    print(real_time_keyword)
    
driver.quit()

벨라루스
본 레거시
산호비녀
지상
쇼미더머니8
쇼미더머니9
맨유 브라이튼
이진이
로또 930회
유지상
